In [34]:
#Importing Packages
import torch
import torch.optim as optim
import torch.nn as nn
from torch.utils.data import TensorDataset, DataLoader

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import sklearn.preprocessing as pre
from sklearn.preprocessing import StandardScaler
import os

In [17]:
#Move to CUDA if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

Using device: cuda


In [18]:
#Importing data from UNSW_NB15
train_df = pd.read_csv('data/UNSW_NB15_training-set.csv')
test_df = pd.read_csv('data/UNSW_NB15_testing-set.csv')

In [19]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 175341 entries, 0 to 175340
Data columns (total 45 columns):
 #   Column             Non-Null Count   Dtype  
---  ------             --------------   -----  
 0   id                 175341 non-null  int64  
 1   dur                175341 non-null  float64
 2   proto              175341 non-null  object 
 3   service            175341 non-null  object 
 4   state              175341 non-null  object 
 5   spkts              175341 non-null  int64  
 6   dpkts              175341 non-null  int64  
 7   sbytes             175341 non-null  int64  
 8   dbytes             175341 non-null  int64  
 9   rate               175341 non-null  float64
 10  sttl               175341 non-null  int64  
 11  dttl               175341 non-null  int64  
 12  sload              175341 non-null  float64
 13  dload              175341 non-null  float64
 14  sloss              175341 non-null  int64  
 15  dloss              175341 non-null  int64  
 16  si

In [20]:
test_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 82332 entries, 0 to 82331
Data columns (total 45 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   id                 82332 non-null  int64  
 1   dur                82332 non-null  float64
 2   proto              82332 non-null  object 
 3   service            82332 non-null  object 
 4   state              82332 non-null  object 
 5   spkts              82332 non-null  int64  
 6   dpkts              82332 non-null  int64  
 7   sbytes             82332 non-null  int64  
 8   dbytes             82332 non-null  int64  
 9   rate               82332 non-null  float64
 10  sttl               82332 non-null  int64  
 11  dttl               82332 non-null  int64  
 12  sload              82332 non-null  float64
 13  dload              82332 non-null  float64
 14  sloss              82332 non-null  int64  
 15  dloss              82332 non-null  int64  
 16  sinpkt             823

In [6]:
# #Encoding Labels in Train Dataset
#
# # train_df['attack_cat'] = train_df['attack_cat'].map(
# #     {
# #   "Analysis": 0,
# #   "Backdoor": 1,
# #   "DoS": 2,
# #   "Exploits": 3,
# #   "Fuzzers": 4,
# #   "Generic": 5,
# #   "Normal": 6,
# #   "Reconnaissance": 7,
# #   "Shellcode": 8,
# #   "Worms": 9
# # }
# # )
#
# train_df = pd.DataFrame({'attack_cat': ["Analysis", "Backdoor", "Normal", "Exploits", "Shellcode", "Fuzzers", "Generic", "Reconnaissance", "DoS", "Worms"]})
#
# le = pre.LabelEncoder()
# train_df['attack_cat_encoded'] = le.fit_transform(train_df['attack_cat'])


In [7]:
# #Encoding Labels in Test Dataset
# test_df = pd.DataFrame({'attack_cat': ["Analysis", "Backdoor", "Normal", "Exploits", "Shellcode", "Fuzzers", "Generic", "Reconnaissance", "DoS", "Worms"]})
#
# le = pre.LabelEncoder()
# test_df['attack_cat_encoded'] = le.fit_transform(test_df['attack_cat'])

In [35]:
#Collecting required features
feature_columns = ['dur', 'spkts', 'dpkts', 'sbytes', 'dbytes', 'rate', 'sload', 'dload', 'smean', 'dmean',]# 'attack_cat_encoded'

X_train = train_df[feature_columns].values
y_train = train_df['label'].values


le = pre.LabelEncoder()
train_df['attack_cat_encoded'] = le.fit_transform(train_df['attack_cat'])

X_test = test_df[feature_columns].values
y_test = test_df['label'].values

In [36]:
#Converting to tensor
scaler = StandardScaler()

X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Converting to tensor and moving to the selected device

X_train_tensor = torch.tensor(X_train_scaled, dtype=torch.float32).unsqueeze(1).to(device)
y_train_tensor = torch.tensor(y_train, dtype=torch.long).to(device)

X_test_tensor = torch.tensor(X_test_scaled, dtype=torch.float32).unsqueeze(1).to(device)
y_test_tensor = torch.tensor(y_test, dtype=torch.long).to(device)

In [37]:
BATCH_SIZE = 32
train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)

test_dataset = TensorDataset(X_test_tensor, y_test_tensor)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False)

print("Data loaded from CSV and prepared for PyTorch.")
print("Training data shape:", X_train_tensor.shape)
print("Test data shape:", X_test_tensor.shape)
print(f"Number of unique labels (classes): {len(np.unique(y_train))}")


Data loaded from CSV and prepared for PyTorch.
Training data shape: torch.Size([175341, 1, 10])
Test data shape: torch.Size([82332, 1, 10])
Number of unique labels (classes): 2


In [38]:
class Net1DCNN(nn.Module):
    def __init__(self, input_channels, sequence_length, num_classes):
        super(Net1DCNN, self).__init__()

        self.conv1 = nn.Conv1d(in_channels=input_channels, out_channels=32, kernel_size=3, padding=1)
        self.relu1 = nn.ReLU()
        self.pool1 = nn.MaxPool1d(kernel_size=2, stride=2)

        self.conv2 = nn.Conv1d(in_channels=32, out_channels=64, kernel_size=3, padding=1)
        self.relu2 = nn.ReLU()
        self.pool2 = nn.MaxPool1d(kernel_size=2, stride=2)


        # self.fc1 = nn.Linear(160 * 1, 64)
        # self.fc2 = nn.Linear(64, num_classes)

        with torch.no_grad():
            dummy_input = torch.randn(1, input_channels, sequence_length)
            dummy_output = self.pool1(self.relu1(self.conv1(dummy_input)))
            dummy_output = self.pool2(self.relu2(self.conv2(dummy_output)))
            flattened_size = dummy_output.view(dummy_output.size(0), -1).shape[1]

        self.fc1 = nn.Linear(flattened_size, 128)
        self.relu_fc = nn.ReLU()
        self.fc2 = nn.Linear(128, num_classes)

    def forward(self, x):
        x = self.conv1(x)
        x = self.relu1(x)
        x = self.pool1(x)

        x = self.conv2(x)
        x = self.relu2(x)
        x = self.pool2(x)

        x = x.view(x.size(0), -1)
        x = self.fc1(x)
        x = self.relu_fc(x)
        x = self.fc2(x)
        return x


In [54]:
input_channels = X_train_tensor.shape[1]
sequence_length = X_train_tensor.shape[2]
num_classes = len(np.unique(y_train))

model = Net1DCNN(input_channels=input_channels, sequence_length=sequence_length, num_classes=num_classes).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.0005)

num_epochs = 50

print("\nStarting model training...")
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {running_loss/len(train_loader):.4f}")

print("Training finished.")



Starting model training...
Epoch [1/50], Loss: 0.2677
Epoch [2/50], Loss: 0.2069
Epoch [3/50], Loss: 0.1879
Epoch [4/50], Loss: 0.1758
Epoch [5/50], Loss: 0.1668
Epoch [6/50], Loss: 0.1615
Epoch [7/50], Loss: 0.1585
Epoch [8/50], Loss: 0.1560
Epoch [9/50], Loss: 0.1543
Epoch [10/50], Loss: 0.1520
Epoch [11/50], Loss: 0.1508
Epoch [12/50], Loss: 0.1502
Epoch [13/50], Loss: 0.1488
Epoch [14/50], Loss: 0.1472
Epoch [15/50], Loss: 0.1459
Epoch [16/50], Loss: 0.1452
Epoch [17/50], Loss: 0.1443
Epoch [18/50], Loss: 0.1436
Epoch [19/50], Loss: 0.1430
Epoch [20/50], Loss: 0.1425
Epoch [21/50], Loss: 0.1413
Epoch [22/50], Loss: 0.1411
Epoch [23/50], Loss: 0.1405
Epoch [24/50], Loss: 0.1403
Epoch [25/50], Loss: 0.1394
Epoch [26/50], Loss: 0.1391
Epoch [27/50], Loss: 0.1382
Epoch [28/50], Loss: 0.1377
Epoch [29/50], Loss: 0.1375
Epoch [30/50], Loss: 0.1370
Epoch [31/50], Loss: 0.1364
Epoch [32/50], Loss: 0.1359
Epoch [33/50], Loss: 0.1354
Epoch [34/50], Loss: 0.1353
Epoch [35/50], Loss: 0.1346
E

In [55]:
model.eval()
correct = 0
total = 0
with torch.no_grad():
    for inputs, labels in test_loader:
        # Move inputs and labels to the device for each batch
        inputs, labels = inputs.to(device), labels.to(device)

        outputs = model(inputs)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

accuracy = 100 * correct / total
print(f"\nAccuracy on the test set: {accuracy:.2f}%")


Accuracy on the test set: 84.30%


In [ ]:
model_dir = 'models'
os.makedirs(model_dir, exist_ok=True)
model_path = os.path.join(model_dir, '1d_cnn_model.pth')


torch.save(model.state_dict(), model_path)
print(f"\nModel saved to {model_path}")